In [1]:
import os
import re
import string
import pandas as pd
import tables
import numpy as np
from datetime import datetime


# Generate a dataframe of dummy data for testing the code
# intentionally randomises length to confirm pd.merge function works as expected
def dummydata(run):
    dummywavelength = list(np.arange(run['wavelength_range'][0], #start
                                    run['wavelength_range'][1],  #stop
                                    run['wavelength_range'][2])) #step

    #Intentionally remove some data points from the end of the list
    missing_samples = np.random.randint(1,10)
    dummywavelength = dummywavelength[:-missing_samples]

    size = len(dummywavelength)
    dummydata = list(np.random.random_sample(size))
    dummycsv = {'wavelength' : dummywavelength, 'transmission' : dummydata}
    df = pd.DataFrame(data=dummycsv, dtype=np.float32)
    return df

# Write dataframe and a metadata object (e.g. dictionary) to an hdf5 file
# Fixed format gives a much smaller file size 
# Table format would allow appending to individual tables, not needed.
def h5store(filename, hdfkey, df, metadata):
    store = pd.HDFStore(filename, mode='a')
    if hdfkey in store:
        store.close()
        raise ValueError(F'Error, node already exists in HDF5 file: {hdfkey}')
    else:
        store.put(hdfkey, df, format='fixed')
        store.get_storer(hdfkey).attrs.metadata = metadata
    store.close()

# Load a dataframe from an hdf5 file, also return associated metadata
def h5load(filename, hdfkey):
    store = pd.HDFStore(filename, mode='r')
    data = store[hdfkey]
    metadata = store.get_storer(hdfkey).attrs.metadata
    store.close()
    return data, metadata

# Returns a list of element indicies for a X by Y sensor array
# e.g. 2x2 would give [A01, A02, B01, B02]
def get_element_list(rows, cols):
	row_list = range(rows)
	col_list = list(string.ascii_uppercase[:cols])
	element_list = []
	for x, y in [(x,y) for x in col_list for y in row_list]:
		element_list.append(F"{x}{y:02d}")
	return element_list

# Extracts dataframes from file and merges them into a single dataframe
# nodelist is a list of strings describing hdf paths
# Outer join means that rows from all dataframes are preserved, and NaN is
# filled where needed
def merge_results(filename, nodelist):
    result = []
    for node in nodelist:
        df = pd.read_hdf(filename, node)
        if len(result) > 0:
            result = pd.merge(result, df, how='outer', on='wavelength')
        else:
            result = df
    return result

# Returns a list of nodes with an attribute called metadata
# Intended as an initial filter to remove irrelevant nodes (or sub nodes)
def get_nodes_with_metadata(filename, hdfkey):
    with tables.open_file(filename, 'r') as f:
        nodelist = []
        for node in f.walk_nodes(hdfkey):
            if hasattr(node._v_attrs, 'metadata'):
                nodelist.append(node._v_pathname)
    return nodelist

# Checks each node in a list for a key/value pair in the metadata.
# Returns a filtered list containing only nodes with desired metadata.
# Can be called recursively to filter for more than one key
def filter_by_metadata(filename, key, value, nodelist):
    with tables.open_file(filename, 'r') as f:
        result = []
        for node in nodelist:
            node = f.get_node(node)
            try:
                if (node._v_attrs.metadata[key] == value ):
                    result.append(node._v_pathname)
            except Exception as e:
                pass
    return result



## Define Run Parameters and Metadata

In [2]:
instrument = {
    'Name'              : 'HAN24',
    'Element_rows'      : 4,
    'Element_cols'      : 4,   
    'Light Source'      : 'Stellarnet LED White',
    'Spectrometer'      : 'Stellarnet BlueWave VIS-25'
}

metadata = {
    'timestamp'         : False,
    'element_index'     : False,
    'hidden'            : False
}

# Set up parameters describing a run of measurements
# Affects how much data is generated in the dummy data function
run = {
    'filename'          : 'test.hdf5',
    'fluid_list'        : ['water', 'beer1', 'beer2'],
    'wavelength_range'  : [400, 420, 0.5], #start, stop, step
    'instrument'        : instrument,
    'elements'          : 'all', #Alternatively, a list of element names
    'repeats'           : 3,
    'measuredOn'        : False
}


## Import from .txt or .csv files

In [8]:
import_dir = "/Users/calum/git/Glasgow/sampleData/Beer x Bitter"

run_date = datetime.utcnow().strftime('%Y_%m_%d')
import_date = datetime.utcnow().strftime('%Y_%m_%d')

# Create a regex to extract metadata from the filename
# example 'HAN24_Sensor9_BeerBitter3_Rotation2.txt'
regex = '(?P<sensor>.+)_Sensor(?P<element_index>.+)_(?P<fluid>.+)_Rotation(?P<rotation>.+).txt'

# specify if the file is tab separated or comma separated 
separator = '\t' #tab

if not os.path.exists(import_dir):
    print("Error, import folder not found")

h5file = run['filename']
if os.path.exists(h5file):
    os.remove(h5file)

for filename in sorted(os.listdir(import_dir)):
    
    match = re.search(regex, filename)
    if not match:
        print(F"Warning regex not matched on filename: {filename}")
        continue

    # Create a metadata dictionary with info extracted from filename
    metadata = match.groupdict()

    # If the element looks like an integer,
    # convert to a string with zero padding
    try: 
        e = int(metadata['element_index'])
        metadata['element_index'] = F"{e:02d}"
    except ValueError:
        #Otherwise, don't modify it
        pass

    # Shorthand for some metadata values, to be used in Fstrings
    s = metadata['sensor']
    f = metadata['fluid']
    e = metadata['element_index']

    # Read the file contents into a dataframe
    df = pd.read_csv(os.path.join(import_dir, filename),sep=separator)
    
    # Check how many repeats exist in the file and label them
    # Assumes the first column represents 'wavelength'
    reps = len(df.columns)-1
    col_names = ['wavelength']
    for r in range(reps):
        col_names.append(str(r+1))
    df.columns = col_names

    # For every repeat in the file, create a new dataframe with only that data
    for r in range(reps):
        df_single = df.filter(['wavelength', str(r+1)])

        # update metadata to preserve info about the imported file
        if 'rotation' in metadata:
            metadata['imported_as'] = F"Rotation{metadata['rotation']}_{r+1}"
            metadata.pop('rotation', None)

        # This loop tries to save the dataframe and metadata to the hdf5
        # If the hdfkey already exists, the repeat number is incremented before
        # retrying
        hdf_r = r+1
        while True:
            try:
                r_str = F"{hdf_r:02d}"
                hdfkey = F"{s}/_{import_date}/{f}/_{e}_rep{r_str}"
                df_single.columns=(['wavelength', F"{f}_rep{r_str}"])
                metadata['repeat'] = r_str
                h5store(h5file, hdfkey, df_single, metadata)
                # print(F"importing {filename} to {hdfkey}")
                break
            except ValueError as err:
                hdf_r += 1

Warning regex not matched on filename: .DS_Store


## Save Dummy Data for all elements into the HDF5 file


In [4]:
if os.path.exists(run['filename']):
    os.remove(run['filename'])

if run['elements'] == 'all':
    elements = get_element_list(instrument['Element_rows'],instrument['Element_cols'])
else:
    elements = run['elements']

date = datetime.utcnow().strftime('%Y_%m_%d')
run['measuredOn'] = date

metadata['instrument'] = instrument['Name']

for f in run['fluid_list']:
    metadata['fluid'] = str(f)

    for e in elements:
        metadata['element_index'] = str(e)

        for r in range(run['repeats']):
            rep = r+1 # Start counting at 1 not 0
            metadata['repeat'] = str(rep)

            hdfkey = F"{instrument['Name']}/_{date}/{f}/{e}_rep{rep}"
            df = dummydata(run)
            df.rename(columns={"transmission" : F"{f}_rep{rep}"}, inplace=True)
            metadata['timestamp'] = datetime.timestamp(datetime.now())
            h5store(run['filename'], hdfkey, df, metadata)


## Inspect an HDF5 file by metadata

In [5]:
filename = run['filename']
measurements = get_nodes_with_metadata(filename, '/')

# measurements = filter_by_metadata(filename, 'element_index', "A00", measurements)
# measurements = filter_by_metadata(filename, 'fluid', "water", measurements)

num = len(measurements)
fluids = set()
elements = set()
dates = set()
data_lengths = set()
reps = set()

for node in measurements:
	data, metadata = h5load(filename, node)
	fluids.add(metadata['fluid'])
	elements.add(metadata['element_index'])
	reps.add(metadata['repeat'])
	try:
		time = datetime.fromtimestamp(metadata['timestamp'])
		dates.add(time.strftime('%Y_%m_%d'))
	except KeyError:
		#No timestamp in file
		pass


	transmission_col = data.columns[1]
	data_len = len(data[transmission_col])

	#Make sure each wavelength has a data point
	if data_len != len(data['wavelength']):
		print("Warning column/index length mismatch")

	data_lengths.add(data_len)
	
expected = len(fluids) * len(elements) * len(reps)

print(F"Found {num} measurements out of {expected} expected, including:\n"
	  F"{len(fluids)} fluids {sorted(fluids)}\n"
	  F"{len(elements)} elements {sorted(elements)}\n"
	  F"{len(reps)} repeats {sorted(reps)}\n"
	  F"{len(dates)} dates {sorted(dates)}\n"
	  F"{len(data_lengths)} Data lengths {sorted(data_lengths)}")


Found 144 measurements out of 144 expected, including:
3 fluids ['beer1', 'beer2', 'water']
16 elements ['A00', 'A01', 'A02', 'A03', 'B00', 'B01', 'B02', 'B03', 'C00', 'C01', 'C02', 'C03', 'D00', 'D01', 'D02', 'D03']
3 repeats ['1', '2', '3']
1 dates ['2021_10_20']
9 Data lengths [31, 32, 33, 34, 35, 36, 37, 38, 39]


## Example of searching by metadata

In [6]:

filename = run['filename']
measurements = get_nodes_with_metadata(filename, '/')
elements = get_element_list(instrument['Element_rows'],instrument['Element_cols'])

A00 = filter_by_metadata(filename, 'element_index', "A00", measurements)
A00_water = filter_by_metadata(filename, 'fluid', "water", A00)

print(A00_water)


['/HAN24/_2021_10_20/water/A00_rep1', '/HAN24/_2021_10_20/water/A00_rep2', '/HAN24/_2021_10_20/water/A00_rep3']


## Export Table

In [7]:

filename = run['filename']
measurements = get_nodes_with_metadata(filename, '/')
# elements = get_element_list(instrument['Element_rows'],instrument['Element_cols'])
# elements = ['A00', 'A01']

elements = set()
for node in measurements:
	data, metadata = h5load(filename, node)
	elements.add(metadata['element_index'])
elements = sorted(elements)

frames=[]
for e in elements:
    selection = filter_by_metadata(filename, 'element_index', e, measurements)
    element_df = merge_results(filename, selection)
    element_df = element_df.transpose()
    iterables = [[F"Sensor {e}"], element_df.loc['wavelength']]
    col_ix = pd.MultiIndex.from_product(iterables)
    element_df.columns = col_ix
    frames.append(element_df)


exportframe = pd.concat(frames, axis=1)
exportframe.drop('wavelength', inplace=True)
exportframe.to_csv('export.csv')
exportframe

Sensor A00                                                    \
wavelength      400.0     400.5     401.0     401.5     402.0     402.5   
beer1_rep1   0.320791  0.314512  0.439127  0.315200  0.828063  0.101104   
beer1_rep2   0.521167  0.578265  0.526919  0.849993  0.745393  0.747178   
beer1_rep3   0.751272  0.190521  0.644996  0.090687  0.096059  0.385306   
beer2_rep1   0.234200  0.963285  0.329422  0.226096  0.106134  0.078250   
beer2_rep2   0.060856  0.003040  0.111737  0.456809  0.986200  0.352483   
beer2_rep3   0.738026  0.999141  0.866252  0.088678  0.615682  0.860215   
water_rep1   0.232566  0.835552  0.891261  0.299803  0.880217  0.517623   
water_rep2   0.652826  0.573642  0.850299  0.167579  0.905165  0.030247   
water_rep3   0.687568  0.068991  0.331174  0.476346  0.415653  0.578587   

                                                    ... Sensor D03            \
wavelength     403.0     403.5     404.0     404.5  ...      413.5     414.0   
beer1_rep1  0.565913  0.152181  0.293608  0.551580  ...   0.127160  0.596455   
beer1_rep2  0.619990  0.091269  0.944231  0.185499  ...   0.820818  0.789384   
beer1_rep3  0.665825  0.391883  0.949542  0.496420  ...   0.708088  0.129466   
beer2_rep1  0.436046  0.925992  0.778308  0.212848  ...   0.132833  0.663853   
beer2_rep2  0.153223  0.394911  0.896166  0.797601  ...   0.701031  0.675646   
beer2_rep3  0.802174  0.526101  0.918270  0.401757  ...   0.802424  0.708551   
water_rep1  0.222726  0.329869  0.418722  0.003137  ...   0.684950  0.516417   
water_rep2  0.852531  0.383824  0.859917  0.186345  ...   0.201142  0.046643   
water_rep3  0.981923  0.263206  0.469289  0.121791  ...   0.263322  0.833790   

                                                                        \
wavelength     414.5     415.0     415.5     416.0     416.5     417.0   
beer1_rep1  0.751341  0.913378  0.972865  0.202945  0.231074  0.517979   
beer1_rep2  0.874701  0.945652  0.248666  0.626864  0.490283  0.662438   
beer1_rep3  0.835567  0.474567  0.234112  0.130536  0.592339  0.274549   
beer2_rep1  0.088058  0.148197       NaN       NaN       NaN       NaN   
beer2_rep2  0.213977  0.630889  0.589496       NaN       NaN       NaN   
beer2_rep3  0.374795  0.638396  0.141146  0.109268  0.818825  0.743289   
water_rep1  0.921937  0.601660       NaN       NaN       NaN       NaN   
water_rep2  0.642714  0.357424  0.167022  0.546353  0.030466  0.871890   
water_rep3  0.227319  0.309859  0.613015  0.390712       NaN       NaN   

                                
wavelength     417.5     418.0  
beer1_rep1  0.175811  0.985120  
beer1_rep2  0.891372       NaN  
beer1_rep3  0.997192       NaN  
beer2_rep1       NaN       NaN  
beer2_rep2       NaN       NaN  
beer2_rep3       NaN       NaN  
water_rep1       NaN       NaN  
water_rep2  0.715299  0.343769  
water_rep3       NaN       NaN  

[9 rows x 617 columns]